In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import os
import csv

url = f"https://programmers.co.kr/job?min_salary=&min_career=&min_employees=&order=recent&page=1"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(url)

time.sleep(1)

def get_last():
    pages = driver.find_elements_by_class_name("page-link")
    last = int(pages[-2].text)
    return last
def get_urls(last):
    links=[]
    for page in range(1, last+1):
      urls = driver.get(f"https://programmers.co.kr/job?min_salary=&min_career=&min_employees=&order=recent&page={page}")
      time.sleep(5)
      print(f"{page} 스크랩 중")
  #         html = driver.page_source
  #         soup = bs4.BeautifulSoup(html,"html.parser")
  #         hrefs = soup.select(.position-title).get['href']
      js = 'return [...document.querySelectorAll(".position-title a")].map(a=>a.href)'
      hrefs = driver.execute_script(js)

      for href in hrefs:
        link = href
        links.append(link)
    return links
def get_info(links):
    recruit=[]
    infomations = []
    stackinfo = []
    for link in links:
      url = link
      driver.get(url)
      time.sleep(3)
      company = driver.find_elements_by_xpath("//h4[@class='sub-title'][1]")
      notice = driver.find_elements_by_xpath("//h2[@class='title']")
      stacks = driver.find_elements_by_xpath("//tr[@class='heavy-use']//td//code")
      index = driver.find_elements_by_xpath("//tbody//tr//td[@class='t-label']")
      content = driver.find_elements_by_xpath("//td[@class='t-content']")
      for c,n in zip(company,notice):
        company = c.text
        notice = n.text
      for i,c in zip(index,content):
        info = i.text
        content = c.text
        doc2 = {
            info:content
        }
        infomations.append(doc2)
      for s in stacks:
        stackinfo.append(s.text)
        doc = {
          'company':company,
          'notice':notice,
          'information':infomations,
          'stacks':stackinfo,
        }
        recruit.append(doc) 
    return recruit

In [ ]:
result = get_urls(get_last())

In [ ]:
result_info = get_info(result)

In [ ]:
links = ['https://programmers.co.kr/job_positions/7126','https://programmers.co.kr/job_positions/7128']
def get_info2():
  recruit=[]
  infomations = []
  stackinfo = []
  for get_url in links[:3]:
    link = get_url
    driver.get(link)
    time.sleep(3)
    company = driver.find_elements_by_xpath("//h4[@class='sub-title'][1]")
    notice = driver.find_elements_by_xpath("//h2[@class='title']")
    
    # stacks = driver.find_elements_by_xpath("//tr[@class='heavy-use']//td//code")
    # index = driver.find_elements_by_xpath("//tbody//tr//td[@class='t-label']")
    # content = driver.find_elements_by_xpath("//td[@class='t-content']")
    for c,n in zip(company,notice):
      company = c.text
      notice = n.text
      index = driver.find_elements_by_xpath("//tbody//tr//td[@class='t-label']")
      content = driver.find_elements_by_xpath("//td[@class='t-content']")
      stacks = driver.find_elements_by_xpath("//tr[@class='heavy-use']//td//code")
      for i,c in zip(index,content):
        info = i.text
        content = c.text
        doc2 = {
            info:content
        }
        infomations.append(doc2)
      for s in stacks:
        stackinfo.append(s.text)
        doc = {
          'company':company,
          'notice':notice,
          'information':infomations,
          'stacks':stackinfo,
        }
        recruit.append(doc) 
  return recruit



In [ ]:
import json

In [ ]:
def toJson(recruit):
    with open('recruit.json', 'w', encoding='utf-8') as file :
        json.dump(get_info(result), file, ensure_ascii=False, indent='\t')


toJson(get_info(result))